In [88]:
import requests
import csv
import pandas as pd
import json
import dateutil
import time
import json

rest_api_domain = "https://api-fxpractice.oanda.com"

def get_token():
    f = open('../.practice_token', 'r')
    return str(f.read().strip())

#access_token = "";

access_token = get_token()
#print(access_token)
account_id = "101-004-7361006-006"
granularity = "D"
instrument = "GBP_USD"
number_of_candles = "180"
candles_endpoint = f'/v3/instruments/{instrument}/candles?count={number_of_candles}&granularity={granularity}'
authorization_header = f'Bearer {access_token}'
trades_endpoint = f"/v3/accounts/{account_id}/trades"


In [89]:
# Docs: http://developer.oanda.com/rest-live-v20/trade-ep/
# For analysis of closed trades: '?instrument=AUD_USD&state=CLOSED&count=500'
def get_trades():
    global rest_api_domain, trades_endpoint
    url = rest_api_domain + trades_endpoint + "?count=500"
    print(authorization_header)
    response = requests.get(url, headers={'Authorization': str(authorization_header)})
    response_json = response.json()
    trades = response_json['trades']
    return trades

In [90]:
def trades_by_instrument(instrument, state='OPEN'): #open or closed
    global rest_api_domain, trades_endpoint
    # Should not do a call per instrument
    url = rest_api_domain + trades_endpoint + f"?instrument={instrument}&state={state}&count=500"
    response = requests.get(url, headers={'Authorization': authorization_header})
    response_json = response.json()
    trades = response_json['trades']
    return trades

In [57]:
def datetime_to_utc_timestamp(date_time):
    return int(time.mktime(date_time.timetuple()))

In [58]:
# receives the time as 'openTime', 'closedTime', 'time' from trade/candle data from Oanda and gets back the unix utc
def time_in_response_to_utc(time_in_response):
    parsed_date_time = dateutil.parser.parse(time_in_response)
    date_time_utc = datetime_to_utc_timestamp(parsed_date_time)
    return date_time_utc

In [126]:
def financing_per_instrument(instrument, state='OPEN'):
    trades = trades_by_instrument(instrument=instrument, state=state)
    financing_dict = { 'financing': sum(float(trade['financing']) for trade in trades), 'instrument': instrument, 'trades': len(trades)}
    financing_dict['fin_trade'] = float(financing_dict['financing']/financing_dict['trades'])
    return financing_dict

In [127]:
instruments = ["NZD_USD", "AUD_USD", "USD_MXN", "GBP_JPY", "GBP_USD", "AUD_JPY", "USD_JPY", "EUR_JPY", "EUR_USD"]
state = "OPEN"

financings = [ financing_per_instrument(instr, state) for instr in instruments]




In [128]:
financings

[{'financing': -27.370099999999997,
  'instrument': 'NZD_USD',
  'trades': 11,
  'fin_trade': -2.4881909090909087},
 {'financing': -53.19259999999999,
  'instrument': 'AUD_USD',
  'trades': 14,
  'fin_trade': -3.799471428571428},
 {'financing': 0.12460000000000004,
  'instrument': 'USD_MXN',
  'trades': 2,
  'fin_trade': 0.06230000000000002},
 {'financing': 3.0878,
  'instrument': 'GBP_JPY',
  'trades': 9,
  'fin_trade': 0.3430888888888889},
 {'financing': -21.093400000000003,
  'instrument': 'GBP_USD',
  'trades': 3,
  'fin_trade': -7.031133333333334},
 {'financing': 13.417300000000001,
  'instrument': 'AUD_JPY',
  'trades': 11,
  'fin_trade': 1.2197545454545455},
 {'financing': 3.0542,
  'instrument': 'USD_JPY',
  'trades': 3,
  'fin_trade': 1.0180666666666667},
 {'financing': -9.879,
  'instrument': 'EUR_JPY',
  'trades': 8,
  'fin_trade': -1.234875},
 {'financing': -25.7361,
  'instrument': 'EUR_USD',
  'trades': 5,
  'fin_trade': -5.14722}]

In [62]:
def extract_candlesticks(response_json):
    candlesticks = []
    for candle in response_json['candles']:
        p_open = float(candle['mid']['o'])
        p_close = float(candle['mid']['c'])
        p_low = float(candle['mid']['l'])
        p_high = float(candle['mid']['h'])
        the_time = candle['time']
        parsed_date_time = dateutil.parser.parse(the_time)
        date_time_utc = datetime_to_utc_timestamp(parsed_date_time)
        candlestick = { 'open': p_open, 'close': p_close, 'low': p_low, 'high': p_high, 'time': date_time_utc}
        candlesticks.append(candlestick)
    return {'instrument': response_json['instrument'], 'granularity': response_json['granularity'], 'candles': candlesticks}
    

In [63]:
def load_candles_as_dict(json_data):
    return json.loads(json_data)['candles']

def read_json_file(filepath):
    with open(filepath, 'r') as thefile:
        data = thefile.read()
        return load_candles_as_dict(data)

In [64]:
#the_time = r['candles'][0]['time']
#print(the_time)
#print(time.mktime(int(the_time)))
#print(time.strftime("%x", time.gmtime(int(the_time))))

In [65]:
def format_date(date_as_timestamp):
    return time.strftime("%x", time.gmtime(int(date_as_timestamp)))

In [66]:
def write_to_json(json_data_as_dict):
    with open('/Users/david/Desktop/json_data.json', 'w') as fp:
        json.dump(json_data_as_dict, fp)

In [67]:
def write_to_csv(candles):
     with open('/Users/david/Desktop/csv_data1.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile,delimiter=',')
        for candle in candles:
            #print(f'Candle:' + str(candle['close']))
            date = format_date(candle['time'])
            data_to_write = [date,candle['close']]
            print(data_to_write)
            spamwriter.writerow(data_to_write)
        

In [68]:
candles = read_json_file('/Users/david/Desktop/json_data.json')

In [69]:
def read_as_pandas_df(trades_json):
    df = pd.read_json(json.dumps(trades_json))
    return df

In [70]:
def parse_trade_from_response(trade, for_open_trades=True):
    trade_list = []
    instrument = trade['instrument']
    open_time = trade['openTime']
    units = int(trade['initialUnits'])
    direction = "Buy" if units > 0 else "Sell"
    financing = trade['financing']
    realized_pl = trade['realizedPL']
    initial_margin_req = trade['initialMarginRequired']
    open_price = trade['price']
    take_profit = 'Not Set'
    stop_loss = 'Not Set'
    if 'takeProfitOrder' in trade:
        take_profit = trade['takeProfitOrder']['price']
    if 'stopLossOrder' in trade:
        stop_loss = trade['stopLossOrder']['price']
    if for_open_trades:
        margin_used = trade['marginUsed']
        unrealized_pl = trade['unrealizedPL']
        trade_list = [instrument, open_time, float(open_price), direction, units, unrealized_pl, take_profit, stop_loss, float(financing)]
    else:
        close_time = trade['closeTime']
        trade_list = [instrument, open_time, close_time, float(open_price), direction, units, take_profit, stop_loss, float(financing)]
    return trade_list

In [71]:
def get_csv_header():
    return [str("Instrument"), str('Open Time'), "Open Price", "Direction","Units", "P/L", "TP", "SL", "Financing"]

In [72]:
def write_csv(filename,rows_as_lists):
    with open(filename, 'w') as csvfile:
        spamwriter = csv.writer(csvfile,delimiter=',')
        for row in rows_as_lists:
            spamwriter.writerow(row)

In [73]:
trades = get_trades()
open_trades = True
all_trades = [ parse_trade_from_response(t,open_trades) for t in trades]

Bearer 07c3f4f198d19e5690ed7e57a317d26b-625b8adfcfd20bef12bf84d9135ac0bf


In [74]:
all_trades = [get_csv_header()] + all_trades

In [75]:
write_csv("/Users/david/Desktop/trades.csv",all_trades)

In [132]:
def summary(trades, state='OPEN'):
    df = read_as_pandas_df(trades)
    df['Direction'] = df.apply(lambda row: "Buy" if int(row['currentUnits']) > 0 else "Sell",axis=1)
    df['currentUnits'] = df.apply(lambda row: -1*int(row['currentUnits']) if int(row['currentUnits']) < 0 else int(row['currentUnits']),axis=1)
    df['openTime'] = df['openTime'].apply(dateutil.parser.parse, dayfirst=False)
    buys = df['currentUnits'][df['Direction'] == 'Buy'].sum()
    sells = df['currentUnits'][df['Direction'] == 'Sell'].sum()
    direction_counts = df['Direction'].value_counts()
    print("Directions:")
    print(direction_counts)
    print("=================")
    print(f"Units bought: {buys}")
    print(f"Units sold: {sells}")
    real_or_unreal = 'unrealizedPL' if state is 'OPEN' else 'realizedPL' 
    df3 = df[['currentUnits','financing',real_or_unreal,'price','instrument','openTime','Direction']]
    return df3
    

In [106]:
# ======== Only for current open trades ======
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
trades = get_trades()
df = read_as_pandas_df(trades)

df['Direction'] = df.apply(lambda row: "Buy" if int(row['currentUnits']) > 0 else "Sell",axis=1)
df['currentUnits'] = df.apply(lambda row: -1*int(row['currentUnits']) if int(row['currentUnits']) < 0 else int(row['currentUnits']),axis=1)
df['openTime'] = df['openTime'].apply(dateutil.parser.parse, dayfirst=False)
buys = df['currentUnits'][df['Direction'] == 'Buy'].sum()
sells = df['currentUnits'][df['Direction'] == 'Sell'].sum()
direction_counts = df['Direction'].value_counts()
print("Directions:")
print(direction_counts)
print("=================")
print(f"Units bought: {buys}")
print(f"Units sold: {sells}")


Bearer 07c3f4f198d19e5690ed7e57a317d26b-625b8adfcfd20bef12bf84d9135ac0bf
Directions:
Buy     64
Sell     3
Name: Direction, dtype: int64
Units bought: 90200
Units sold: 3200


In [112]:
df_grouped = df.groupby(['instrument','Direction'],as_index=False)
df2 = df_grouped.agg({'currentUnits':'sum', 'financing': 'sum', 'realizedPL': 'sum'})
# df2 = df_grouped.agg({'initialUnits':'sum', 'financing': 'sum', 'realizedPL': 'sum'})
# df2['perc'] = df2.apply(lambda row: 100*row['currentUnits']/buys if row['Direction'] == 'Buy' else 100*row['currentUnits']/sells,axis=1)
#df2
df3 = df[['currentUnits','financing','unrealizedPL','price','instrument','openTime','Direction']]
df3 = df3[df3['instrument'] == 'AUD_JPY']
df3


,currentUnits,financing,realizedPL,price,instrument,openTime,Direction
2,1200,0.0361,0,74.520,AUD_JPY,2019-06-17 15:09:19.073070+00:00,Buy
10,1300,0.1156,0,74.814,AUD_JPY,2019-06-13 07:16:00.166038+00:00,Buy
18,2000,0.6334,0,75.761,AUD_JPY,2019-05-27 16:01:44.037591+00:00,Buy
22,1400,0.6956,0,76.156,AUD_JPY,2019-05-14 08:18:26.951172+00:00,Buy
26,1300,0.7220,0,76.667,AUD_JPY,2019-05-09 23:09:29.824122+00:00,Buy
27,1400,0.8180,0,77.183,AUD_JPY,2019-05-07 17:34:16.968629+00:00,Buy
32,1500,0.9689,0,77.949,AUD_JPY,2019-05-03 01:05:05.982322+00:00,Buy
35,1500,0.9947,0,78.235,AUD_JPY,2019-05-01 17:01:13.404586+00:00,Buy
36,1300,1.2593,0,78.687,AUD_JPY,2019-04-24 08:06:27.728765+00:00,Buy
39,1100,1.3227,0,79.864,AUD_JPY,2019-04-22 19:25:24.942045+00:00,Buy


In [133]:
tr = trades_by_instrument('USD_MXN','OPEN')

In [134]:
summary(tr,'OPEN')

Directions:
Sell    1
Buy     1
Name: Direction, dtype: int64
Units bought: 1300
Units sold: 1100


,currentUnits,financing,unrealizedPL,price,instrument,openTime,Direction
0,1300,-1.4904,-5.7942,19.13029,USD_MXN,2019-06-11 19:44:12.897942+00:00,Buy
1,1100,1.6150,5.4346,19.15016,USD_MXN,2019-05-29 18:28:55.978850+00:00,Sell
